# 0. Preliminaries

   ## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from IPython.display import HTML as html_print
import seaborn as sns    # for heatmaps
import scipy.special as sp

import logomaker         # if not present install with: pip install logomaker

## Parameters of the problem

In [ ]:
# These are global variables to be used throughout the notebook
num_sequences = 50
sequence_length = 100
motif_length = 8
max_position = sequence_length - motif_length + 1  # this will be needed in multiple places
alpha = 0.2 # this alpha is used in the MCMC sampling

## Initialize the random number generator

In [ ]:
random_seed = 2023
rng = np.random.default_rng(random_seed)

# 1. Generate and re-estimate motifs

## Functions to generate and view a motif
We will represent the motif by a list length K of probability distributions over [0..3]

In [ ]:
def sample_motif(motif_length=motif_length, alpha=1):
    """
    Create a motif from the dirichlet distribution
    """
    return *** # fill using rng.dirichlet() 

def draw_motif(motif, title="", ax=None):
    """
    Draw a logo of the motif with logomaker
    """
    with warnings.catch_warnings():
        # suppress harmless warnings from logomaker
        warnings.simplefilter("ignore") 
    
        # Make dataframe for logomake
        df = pd.DataFrame(motif, columns=["A", "C", "G", "T"])
        df = logomaker.transform_matrix(df, from_type="probability", to_type="information")
        logo = logomaker.Logo(df, ax=ax)

        logo.ax.set_title(title)
        # logo.ax.set_xlabel("Position")
        logo.ax.set_ylabel("Total information")
        logo.ax.set_ylim(0, np.log2(4))
        logo.fig.show()
        

## Test out motif generation
Get the next cell to run yourself, by fixing the code in sample_motif() function above, then explore what happens as you vary alpha, e.g. to 0.5, 1, 5, perhaps 0.01. Try each value several times.

In [ ]:
M = sample_motif(alpha=0.2)
# print out the motif nicely
with np.printoptions(precision=3, suppress=True):
    print(np.matrix.transpose(M))
# draw the logo for the motif - we will use this function below to visualise motifs
draw_motif(M)

## Next we generate some example instances from a motif
The motifs we just made are probabilistic models of sequences of length K.  Here we generate several sequences from our current motif.  First you need to fill in the missing code in the function definition in the next cell, and run this to define the function. Then you can explore using it to sample instances of the motif. Try this with motifs made with different values of alpha (obtained by re-running the cell above with the desired value of alpha). Motifs made with smaller values of alpha should create more conserved sets of sample instances.

In [ ]:
def sample_sequence_from_motif(motif):
    """
    Sample an instance of the motif from the distribution defined by the motif
    """
    # Initialize the sequence array
    motif_sequence = np.zeros(motif_length, dtype=int)

    # Draw each sequence element from motif
    for k in range(motif_length):
        motif_sequence[k] = *** # fill using rng.choice() - the distribution for site k is stored in motif[k,:]

    return motif_sequence

In [ ]:
motif_sequences = []
for i in range(1000):
    seq = sample_sequence_from_motif(M)
    if(i < 10):
        for s in seq:
            print ("ACGT"[s], end='')
        print()
    motif_sequences.append(seq)

## Re-estimate motif from sequences

Given instances, we can re-estimate the motif by calculating the Bayesian posterior mean.

In [ ]:
def posterior_mean_motif_from_sequences(sequences_list, alpha):
    """
    Estimate the motif by updating the prior Dirichlet distribution with observed sequences.
    """
    # Convert sequences argument from list to numpy array
    sequences = np.array(sequences_list)
    
    # Initialize the motif array - each row corresponds to a position in the motif,
    # with 4 columns per row for the probabilities of the four possible nucleotides
    motif_estimate = np.zeros((motif_length, 4), dtype=float)   
    
    # Add dirichlet prior
    motif_estimate += alpha
    
    # Build the estimate
    for k in range(motif_length):
        for n in range(len(sequences)):
            # Which nucleotide is in the k-th position of the n-th sequence?
            j = sequences[n, k]

            # Update the respective estimate
            motif_estimate[k, j] += 1
                
        # Normalise the motif probabilities, so each motif_estimate[k,:] sums to 1
        ***
        
    return motif_estimate

Here we re-estimate the motif, and calculate the KL divergence between our estimate and the true motif, also plotting them side by side.  How does the KL divergence vary as we change alpha, and also as we change the number of instances used (default :10 in the first line below).

In [ ]:
estimated_motif = posterior_mean_motif_from_sequences(motif_sequences[:10], 1)
print(estimated_motif)
# plot the estimated and true motifs
fig, axs = plt.subplots(1, 2, figsize=(12, 3))
draw_motif(estimated_motif, ax=axs[0])
axs[0].set_title("Estimated motif")
draw_motif(M,ax=axs[1])
axs[1].set_title("True motif")
print("Summed relative entropy (KL divergence): ", sp.rel_entr(M,estimated_motif).sum())

# 2.	Generate long sequences, calculate the log likelihood and find posterior postion distribution

## First, generate a set of long sequences that each incorporate one copy of the motif

In [ ]:
def generate_long_sequences(motif):
    """
    Sample a set of sequences, with a single instance of the motif 
    embedded at a random position in each sequence
    """        
    # Initialize the sequences matrix with random bases
    # This matrix has a row per sequence, and the number of columns is the sequence length. The
    # row contains the sequence encoded into 0,1,2 and 3.
    sequences = rng.integers(4, size=(num_sequences, sequence_length))

    # Draw positions for motifs - each position can be from 0 to sequence_length - motif_length.
    # Draw one position per sequence.
    positions = rng.integers(sequence_length - motif_length + 1, size=num_sequences)

    # Per full sequence, draw from motif and insert
    for n in range(num_sequences):
        # replace the subsequence of length motif_length at positions[n] with a motif instance
        ***
             
    return sequences, positions


def cstr(s, color='black'):
    return "<text style='color:{};font-family:monospace;font-size:1em'>{}</text>".format(color, s)

def draw_sequences(sequences, positions=None):
    """
    Show the sequences in nice text, optionally highlighting the motif positions in red
    """
    text = ""
    for n in range(len(sequences)):
        row = ""
        for s in sequences[n]:
            row += "ACGT"[s]
        row += "\n"
        
        # If no positions to show, everything is black
        if positions is None:                  
            row = cstr(row, color='black')
    
        # Otherwise, color in red
        else:
            pos = positions[n]
            row = \
                cstr(row[:pos], color="black") + \
                cstr(row[pos:pos+motif_length], color="red") + \
                cstr(row[pos+motif_length:], color="black")
        
        text += row
    
    display(html_print(text))


In [ ]:
sequences, positions = generate_long_sequences(M)
draw_sequences(sequences[:20], positions)  # use without positions to show without highlighting

## What is the posterior distribution of a position given a sequence and motif?

In [ ]:
def posterior_position_given_sequence_and_motif(sequence, motif):
    
    # Initialize an array to hold the posterior distribution
    posterior_distribution = np.zeros(max_position, dtype=float)
    
    # For each possible position...
    for i in range(max_position):
        prob = 1.0
        # Calculate the probability of seeing the sequence given the motif
        for k in range(motif_length):
            # Multiply the probability of seeing the nucleotide at the k-th position according to the motif
            prob *= motif[k, sequence[i+k]]
            
        # Save this probability
        posterior_distribution[i] = prob
        
    # Normalize by the sum, to have a proper distribution
    posterior_distribution /= posterior_distribution.sum()
        
    return posterior_distribution

### Test

Try this with motifs and sequences made with different alpha values. But remember to go back to alpha = 0.2 before proceeding to the next section on MCMC.

In [ ]:
pp = posterior_position_given_sequence_and_motif(sequences[2], M)
fig, ax = plt.subplots(figsize=(15, 3));
ax.plot(pp, '.');
ax.set_xlabel("Position in sequence");
ax.set_ylabel("Posterior prob.");

# 3. Markov chain Monte-Carlo (MCMC)

As before we first define the necessary functions, with you having to fill in some missing fields to ensure that you read the code and understand what is being done.

First we implement and test Gibbs sampling.  Then we add shift operations using a Metropolis acceptance test to preserve detailed balance.

## 3.1. Gibbs Sampling

In [ ]:
def log_likelihood(sequences, motif):
    """
    Calculate the log-likehood of the sequences given the motif, 
    integrating over all positions.
    """
    # Initialize
    ll = 0.0
    constant_part = (1.0 / max_position) * (1.0/4)**(sequence_length - motif_length)
    
    # Per sequence, add the log-likelihood
    for n in range(num_sequences):
        pseq = 0.0
        for i in range(max_position):
            # add to pseq the probability of sequence n with motif at position i (several lines needed)
            part = constant_part
            for k in range(motif_length):
                part *= motif[k, sequences[n, i + k]] 
            pseq += part
        ll += np.log(pseq)
                        
    return ll

In [ ]:
def log_likelihood_with_positions(sequences, positions, motif):

    # Initialize
    ll = 0.0
    
    # Per sequence, add the log-likelihood
    for n in range(num_sequences):
        sequence = sequences[n]
        start_position = positions[n]
        
        prob = 1.0
        for k in range(motif_length):
            prob *= motif[k, sequence[start_position + k]] 
            
        ll += np.log(prob)
                        
    return ll

In [ ]:
def gibbs_sample_positions(sequences, estimated_motif):

    # Initialize an array to hold the new positions, per sequence
    positions = np.zeros(len(sequences), dtype=int)
    
    # Per sequence, draw a new position:
    for n in range(num_sequences):
        # Get the posterior distribution over positions, for this sequence
        sequence = sequences[n]
        positions_posterior = posterior_position_given_sequence_and_motif(sequence, estimated_motif)
        
        # Draw the new position from the posterior
        positions[n] = rng.choice(max_position, p=***) # fill in the distribution used to select the position
        
    return positions


In [ ]:
def gibbs_sample_motif(alpha, sequences, positions, motif_length):

    # Calculate the dirichlet parameters
    posterior_parameters = np.ones((motif_length, 4), dtype=float) * alpha    
    for k in range(motif_length):
        for n in range(num_sequences):
            a = sequences[n, positions[n] + k]
            posterior_parameters[k, a] += 1         
    
    # Draw new motif from this distribution
    motif = np.zeros((motif_length, 4), dtype=float)
    for k in range(motif_length):
        motif[k, :] = rng.dirichlet(***) # fill in the parameter set to sample from
        
    return motif
    

In [ ]:
def mcmc_gibbs_sampling(sequences, motif_length, alpha, n_iterations, print_every=None):

    # Initialize the motif estimate randomly with a new sample from a Dirichlet with default (broad) 
    sampled_motif = sample_motif()
    
    # Initialize random positions
    sampled_positions = rng.integers(max_position, size=num_sequences)
        
    # MCMC iterations
    lls = []
    motifs = []
    for n_iter in range(n_iterations):
        # Store current motif and log-likelihood 
        motifs.append(estimated_motif)
        ll = log_likelihood_with_positions(sequences, sampled_positions, sampled_motif)
        lls.append(ll)
        if (print_every is not None) and (n_iter % print_every == 0):
            print(f"Iteration {n_iter}: LL = {ll}")
            
        # Sample new positions
        sampled_positions = gibbs_sample_positions(sequences, sampled_motif)
        
        # Sample new motif
        sampled_motif = gibbs_sample_motif(alpha, sequences, sampled_positions, motif_length)
        
    return sampled_motif, lls, motifs
    

In [ ]:
def plot_results(iters, est_motif, lls, posteriors, motifs, display_true=True):
    if (display_true):
        print (f"LL with true motif {log_likelihood(sequences, M)}")
    print (f"LL with final estimated motif {log_likelihood(sequences, est_motif)}")
       
    if (display_true): # plot the true and final estimated motifs
        fig, axs = plt.subplots(1, 2, figsize=(12, 3))
        draw_motif(M, ax=axs[0])
        axs[0].set_title("True motif")
        draw_motif(est_motif,ax=axs[1])
        axs[1].set_title("Final estimated motif")
    else:
        draw_motif(est_motif,title="Final estimated motif")
    
    if lls is not None:
        # plot the log likelihoods
        plt.figure()
        plt.plot(lls)
        plt.xlabel("# of iteration")
        plt.ylabel("log likelihood")
    
    # plot heatmaps of the motif locations at given iterations
    if len(iters) > 0 and posteriors is not None:
        fig, axs = plt.subplots(1, len(iters), figsize=(20, 8))
        for ax, n_iter in zip(axs, iters):
            sns.heatmap(posteriors[n_iter][:,:], ax=ax, vmin=0, vmax=1);
            ax.set_title(f"Iteration # {n_iter}")
            ax.set_ylabel("Sequence #")
            ax.set_xlabel("Position")
    
    # and below them the motifs
    if len(iters) > 0 and motifs is not None:
        fig, axs = plt.subplots(1, len(iters), figsize=(20, 2))
        for ax, n_iter in zip(axs, iters):
            draw_motif(motifs[n_iter],ax=ax)

## Run the Gibbs sampler several times

Does it always converge? Does it always converge on the correct solution?

How does its convergence depend on the number of sequences, or on value of alpha for the true motif (the cell two down is a handy place to change these values and sample a new motif and set of sequences).

In [ ]:
est_motif, lls, motifs = mcmc_gibbs_sampling(sequences, motif_length, alpha, 200, print_every=50)
plot_results([0,25,50,100,199], est_motif, lls, None, motifs)

In [ ]:
# this is a useful cell of code repeated from above that can be run to change the motif and sequences

M = sample_motif(alpha = 0.2)       # try alpha = 0.2, 0.5, 1, perhaps even more
# if it hasn't converged correctly you could try generating more sequences by uncommenting/changing the next line
num_sequences = 100
sequences, positions = generate_long_sequences(M)

## 3.2. Metropolis

Here we add a Metropolis step to rotate the motif one position left or right.  Note that rotating it allows it to be reversible, helping ensure detailed balance.

This time if we run the MCMC multiple times do we always converge on the same solution?  Are there any negatives to adding the extra step?

In [2]:
def metropolis_step(sequences, estimated_positions, estimated_motif, current_ll, max_position):
    
    max_position = sequence_length - motif_length + 1

    # Make new state
    if rng.random() < 0.5:
        # abcdefgh -> habcdefg
        new_motif = np.roll(estimated_motif, 1, axis=0)
        new_positions = (estimated_positions - 1) % max_position
    else:
        # abcdefgh -> bcdefgha
        new_motif = np.roll(estimated_motif, -1, axis=0)
        new_positions = (estimated_positions + 1) % max_position
        
    # Calculate likelihood of new state
    new_ll = log_likelihood_with_positions(sequences, new_positions, new_motif)
    
    # Acceptance probability
    if new_ll > current_ll:
        acceptance_probability = 1.0
    else:
        acceptance_probability = np.exp(new_ll - current_ll)
    
    # Accept if needed
    if ***:  # insert acceptance test
        return new_motif, new_positions, True
    else:
        return estimated_motif, estimated_positions, False
        

SyntaxError: invalid syntax (788187989.py, line 25)

In [ ]:
def mcmc_metropolis_sampling(sequences, motif_length, alpha, n_iterations, print_every=None):

    rng = np.random.default_rng() # want new random number generator for each run when parallelized below
    
    # Initialize the motif estimate randomly
    estimated_motif = rng.random(size=(motif_length, 4))
    
    # Divide each row by its sum
    for k in range(motif_length):
        total = 0
        for i in range(4):
            total += estimated_motif[k, i]
        for i in range(4):
            estimated_motif[k, i] /= total
    
    # Initialize random positions
    max_position = sequence_length - motif_length + 1
    estimated_positions = rng.integers(max_position, size=num_sequences)
        
    # MCMC iterations
    lls = []
    motifs = []
    all_shifted = []
    for n_iter in range(n_iterations):
        # Print current log-likelihood
        motifs.append(estimated_motif)
        ll = log_likelihood_with_positions(sequences, estimated_positions, estimated_motif)
        lls.append(ll)
        if (print_every is not None) and (n_iter % print_every == 0):
            print(f"Iteration {n_iter}: LL = {ll}")
            
        # Sample new positions
        estimated_positions = gibbs_sample_positions(sequences, estimated_motif)
        
        # Sample new motif
        estimated_motif = gibbs_sample_motif(alpha, sequences, estimated_positions, motif_length)
        
        # Possible shift
        estimated_motif, estimated_positions, shifted = \
            metropolis_step(sequences, estimated_positions, estimated_motif, ll, max_position)
        
        all_shifted.append(shifted)
        
    return estimated_motif, lls, motifs
    

In [ ]:
est_motif, lls, motifs = mcmc_metropolis_sampling(sequences, motif_length, alpha, 200, print_every=50)
plot_results([0,25,50,100,199], est_motif, lls, None, motifs)

# 4. Real data
Here we will explore a real data set of 200 promoter sequences identified in [Serizay et al. (2020)](https://genome.cshlp.org/content/30/12/1752.long) as being active in genes with tissue-specific expression in the intestine of the nematode *C. elegans*.

In [ ]:
# First we upload the data set
# pip install biopython
import Bio.SeqIO

fasta_sequences = Bio.SeqIO.parse(open("elegans_intestine.WS235.fa"),'fasta')

sequences = []
for fasta in fasta_sequences:
    s = str(fasta.seq)
    table = s.maketrans('ACGT','0123')
    s = s.translate(table)
    s = np.array(list(s)).astype(int)
    sequences.append(s)

sequences = np.array(sequences)
num_sequences, sequence_length = sequences.shape
print("num_sequences", num_sequences, "sequence_length", sequence_length)

## Now we can run our MCMC program
Try running it several times.

It is likely that you will get different outcomes. This is because with real data there may be more than one pattern present, unlike in our simulations. Also unlike in our simulations, in real data the background model of a simple 0.25 probability per base is clearly over-simplified.

In [ ]:
# try running this several times
est_motif, lls, motifs = mcmc_metropolis_sampling(sequences, motif_length, alpha, 200, print_every=50)
plot_results([0,25,50,100,199], est_motif, lls, None, motifs, display_true=False)

## Parallel search for high scoring motifs
Because different random starts given different results, we will run 20 jobs in parallel, then sort the results by score.  You may get some runtime warnings in pink lines, which you can ignore.

You will need to wait until you see "Done 20 out of 20" in order for all the jobs to finish, before sorting and printing in the next cell.  How many run at once depends on the computer you are working on.

How many fundamentally different patterns do you find?

In [ ]:
# to find the top matches, run MCMC 20 times and sort the results by the log likelihood

import joblib

results = joblib.Parallel(n_jobs=-1,verbose=100)(   # this uses all available cores to parallelise
    joblib.delayed(mcmc_metropolis_sampling)(sequences, motif_length, alpha, 200)
        for i in range(20)
)

In [ ]:
# store just the final log likelihoods and motifs in list LLmotif, then sort

LLmotif = []
for R in results:
    LLmotif.append([R[1][-1],R[0]])  # the final log likelihood and the motif
from operator import itemgetter
LLmotif.sort(key=itemgetter(0), reverse=True)
for x in LLmotif:
    draw_motif(x[1], title=f"LL {x[0]}")

## Compare to MEME

This is [a link to the output of a MEME run](https://meme-suite.org/meme//opal-jobs/appMEME_5.5.416999974272851417702622/meme.html) on the same sequences, using default parameters in the OOPS (One Occurrence Per Sequence) mode.

Are there similarities, or differences, between your high scoring motifs and the ones found by MEME? You should note that MEME is more sophisticated, searching both strands, with a background model that it learns from the data.

## Search in the JASPAR database using Tomtom

The [JASPAR](https://jaspar.genereg.net/) database contains a curated set of previously established binding motif patterns for hundreds of transcription factors and species, including *C.elegans*.

You can search it to find known patterns similar to motifs you have found using another program in the MEME suite, [TomTom](https://meme-suite.org/meme/tools/tomtom). 

To load your motifs found in from the worm intestine practical into TomTom, you need to first write them out as a file in MEME minimal format. Below we define code to do that, then give instructions to upload this file to the TomTom web site.
    
What does your lead pattern correspond to?  Remember that the sequence you obtain may be the reverse complement of what MEME finds (it searches both strands).

In [ ]:
def write_MEME_file(motifs, file_name="MCB.meme", motif_name="MCB"):
    with open(file_name,'w') as f:
        # write header
        f.write("MEME version 5.5.0\n\n")
        f.write("ALPHABET= ACGT\n\n")
        f.write("strands: + -\n\n")
        # write motifs
        count = 0
        for m in motifs:
            count = count + 1
            f.write(f"MOTIF {motif_name}-{count}\n\n")
            f.write(f"letter-probability matrix: alength= 4 w= {motif_length}\n")
            np.savetxt(f, m, delimiter=' ', fmt='%.4f')
            f.write("\n")
        f.close()
    print(len(motifs), "motifs written")

In [ ]:
write_MEME_file([LLmotif[0][1], LLmotif[2][1]]) # list here the motifs you want, e.g. numbers 0 and 2 in the sorted list

## Search in TomTom
* go to the [TomTom submission link](https://meme-suite.org/meme/tools/tomtom)
* upload the file you have just written (called MCB.meme, unless you renamed it)
* select the "JASPAR (NON-REDUNDANT) DNA" target motif database
* select the "JASPAR CORE (2022) nematodes", unless you want to search all species, in which case leave it as "JASPAR CORE"
* press "Start search"

What do you find?

## Another promoter region file

If you enjoyed that, and have time left, there is another file elegans_muscle.WS235.fa you can try, containing 200 promoter sequences from muscle genes, from the same paper.